Data Extraction and Text Analysis


Importing libraries

In [1]:
!pip install xlsxwriter
!pip install textstat
!pip install excel2json-3

import plotly.graph_objs
import requests
import pandas as pd
from bs4 import BeautifulSoup
import string
import re
import urllib
import textstat
import json
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from textblob import TextBlob

import excel2json
import xlsxwriter


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 105 kB 5.1 MB/s 
     |████████████████████████████████| 2.0 MB 37.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Loading Data

In [3]:
excel2json.convert_from_file('Input.xlsx')

Main Function

 A Function which loops over the URLs which are in the dictionary which is used as a paramter for the fuction

In [5]:
def main(y):
  for k in y:
    url = str(y[k])
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
    r = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(r.content, 'html5lib')

    #Extracts title from the url
    title=soup.find('h1',class_="entry-title")
    title=title.text.replace('\n'," ")

    #Extracts content from url
    content=soup.findAll(attrs={'class':'td-post-content'})
    content=content[0].text.replace('\n'," ")
    content = content.translate(str.maketrans('', '', string.punctuation))

    #Converting content into tokens
    from nltk.tokenize import word_tokenize
    text_tokens = word_tokenize(content)

    #Removing stopwords from the tokens
    my_stop_words = stopwords.words('english')
    my_stop_words.append('the')
    no_stop_tokens = [word for word in text_tokens if not word in my_stop_words]

    #Checking for the positive words using positive-words.txt
    with open("positive-words.txt",'r') as pos_word:
     pos_words = pos_word.read().split("\n")
     pos_words = pos_words[5:]

    pos_count = " ".join([w for w in no_stop_tokens if w in pos_words])
    pos_count = pos_count.split(" ")
    Positive_score=len(pos_count) 
    
    #Checking for the negative words using negative-words.txt
    with open("negative-words.txt","r",encoding = "ISO-8859-1") as neg:
      negwords = neg.read().split("\n")
      negwords = negwords[36:]

    neg_count = " ".join ([w for w in no_stop_tokens if w in negwords])
    neg_count=neg_count.split(" ")
    Negative_score=len(neg_count)
    
    filter_content = ' '.join(no_stop_tokens)

    #Calculating Polarity Score and Subjectivity Score
    sentiment=TextBlob(content).sentiment

    #Average Sentence length
    AVG_SENTENCE_LENGTH = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))

    #FOG Index
    FOG_index=(textstat.gunning_fog(content))

    #Average Number of words per Sentence
    AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
    AVG_NUMBER_OF_WORDS_PER_SENTENCE=sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE)

    #Complex word Count
    def syllable_count(word):
      count = 0
      vowels = "AEIOUYaeiouy"
      if word[0] in vowels:
          count += 1
      for index in range(1, len(word)):
          if word[index] in vowels and word[index - 1] not in vowels:
              count += 1
              if word.endswith("es"or "ed"):
                  count -= 1
      if count == 0:
          count += 1
      return count

    COMPLEX_WORDS=syllable_count(content)

    #Word Counts
    Words_counts=len(content)

    #Percentage of Complex words
    pcw=(COMPLEX_WORDS/Words_counts)*100

    #Average Word Length
    Average_Word_Length=len(content.replace(' ',''))/len(content.split())

    #Syllable per word
    word=content.replace(' ','')
    syllable_count=0
    for w in word:
          if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
                syllable_count=syllable_count+1
    spw = syllable_count/len(content.split())

    #Personal Pronouns
    def ProperNounExtractor(text):
      count = 0
      sentences = nltk.sent_tokenize(text)
      for sentence in sentences:
          words = nltk.word_tokenize(sentence)
          tagged = nltk.pos_tag(words)
          for (word, tag) in tagged:
              if tag == 'PRP':
                count+=1

      return(count)
    Personal_Pronouns=ProperNounExtractor(content)
    
    #Adding individual elements of a row in dataframe
    list =[url,title,content,Positive_score,Negative_score,sentiment.polarity,sentiment.subjectivity, AVG_SENTENCE_LENGTH, FOG_index, AVG_NUMBER_OF_WORDS_PER_SENTENCE, COMPLEX_WORDS, Words_counts, pcw, Average_Word_Length, spw, Personal_Pronouns]

    #Converting str to int
    z= int(k)

    #Adding one list ata atime to the dataframe
    df.loc[z] = list


In [7]:
#Converting excel to json
excel_data_df = pd.read_excel('Input.xlsx', sheet_name='Sheet1')
json_str = excel_data_df.to_json()

#Converting json to dictionary via loads method
x = json.loads(json_str)

#To fetch only URLS
y = x['URL']

#Defining the dataframe
df=pd.DataFrame(columns=['url','title','content','Positive_Score','Negative_Score', 'Polarity', 'Subjectivity', 'AVG_SENTENCE_LENGTH', 'FOG_index', 'AVG_NUMBER_OF_WORDS_PER_SENTENCE', 'COMPLEX_WORDS', 'Words_counts', 'Percentage_of_complex_words', 'Average_Word_Length', 'Syllable_per_word', 'Personal pronouns'])

#Calling the main function
main(y)

#Generation Output file of the dataframe
with pd.ExcelWriter('output.xlsx') as writer:  

    df.to_excel(writer, sheet_name='Sheet1')

